In [3]:

## Copyright (c) 2016 Mikel Bober-Irizar
## Feature-Time Instability Metric
# Script to evaluate the change in properties of features over time
# To use, call the function find_ovalue(FEATURE, TARGET)

import numpy as np
import pandas as pd
import pickle
from multiprocessing import Pool

########## CONFIG #########

# Resolution of time axis to test
time_res = 2
# Resolution of feature value binning
x_res = 200
# Higher values will find smaller fluctuations in data but may have more noise

# Threshold of values that have to be in a histogram bin for it to be considered:
thresh = 0.001

# Method to measure stabilitiy, either 'inter' for histogram intersection or 'purity' for tree split purity
method = 'purity'

# Only used for the purity metric, gives weight to the different splits based on how many samples exist in the bin
weighted = True
ignore_zero = False

######### END CONFIG ##########

def hist_inter(a, b, bins):
    # Uses Histogram intersection distance function to measure instability

    # Find range
    hist_max = max(max(a), max(b))
    hist_min = min(min(a), min(b))

    # np.histogram 'normed' is broken, normalisation must be done manually
    hist_a = np.histogram(a, bins=bins, range=(hist_min, hist_max), normed=False)[0].tolist()
    hist_b = np.histogram(b, bins=bins, range=(hist_min, hist_max), normed=False)[0].tolist()

    # Manual normalisation of histograms
    size_a = len(a)
    size_b = len(b)
    hist_a = [x / size_a for x in hist_a]
    hist_b = [x / size_b for x in hist_b]

    k = 0
    i = 0
    # Evaluate histogram intersection
    for d in zip(hist_a, hist_b):
        if sum(d) > thresh:
                k += min(d)
                i += 1

    return k

def hist_purity(a, b, target_a, target_b, bins, weighted=True, ignore_nan=False):

    # Get range of histogram to use
    hist_max = max(max(a), max(b))
    hist_min = min(min(a), min(b))

    hist = pd.DataFrame()
    hist['a'] = a
    hist['b'] = b
    hist['ta'] = target_a
    hist['tb'] = target_b

    # Separate data into labels
    a_true = hist.loc[hist.ta == 1]['a'].values#.tolist()
    a_false = hist.loc[hist.ta < 1]['a'].values#.tolist()
    b_true = hist.loc[hist.tb == 1]['b'].values#.tolist()
    b_false = hist.loc[hist.tb < 1]['b'].values#.tolist()

    # Compute histograms
    hist_a_true = np.histogram(a_true, bins=bins, range=(hist_min, hist_max), normed=False)[0]
    hist_a_false = np.histogram(a_false, bins=bins, range=(hist_min, hist_max), normed=False)[0]
    hist_b_true = np.histogram(b_true, bins=bins, range=(hist_min, hist_max), normed=False)[0]
    hist_b_false = np.histogram(b_false, bins=bins, range=(hist_min, hist_max), normed=False)[0]

    # Compute split purity
    hist_a_tot = hist_a_true + hist_a_false
    hist_b_tot = hist_b_true + hist_b_false
    hist_a_purity = hist_a_true / hist_a_tot
    hist_b_purity = hist_b_true / hist_b_tot

    if ignore_nan is False:
        hist_a_purity = np.nan_to_num(hist_a_purity)
        hist_b_purity = np.nan_to_num(hist_b_purity)

    # Compute histogram weights
    hist_weights = ((hist_a_true + hist_a_false) / np.sum(hist_a_true + hist_a_false)) + ((hist_b_true + hist_b_false) / np.sum(hist_b_true + hist_b_false)) / 2

    if weighted is False:
        k = np.nansum(np.abs(hist_a_purity - hist_b_purity)) / len(a + b)
    if weighted is True:
        k = np.nansum(np.abs(hist_a_purity - hist_b_purity) * hist_weights)

    #print(k)
    return k

def find_ovalue_inter(feature, target):
    ftr_true = []
    ftr_false = []

    # Separate into positive and negative samples
    for x in zip(feature, target):
        if x[1] >0:
            ftr_true.append(x[0])
        else:
            ftr_false.append(x[0])

    # Split into time bins
    chunks_true = [x.tolist() for x in np.array_split(ftr_true, time_res)]
    chunks_false = [x.tolist() for x in np.array_split(ftr_false, time_res)]

    cross = []
    # Shoddy method for cross-checking chunks
    for x in chunks_true:
        for y in chunks_true:
            if x != y and y > x:
                dist = hist_inter(x, y, x_res)
                cross.append(dist)

    for x in chunks_false:
        for y in chunks_false:
            if x != y and y > x:
                dist = hist_inter(x, y, x_res)
                cross.append(dist)

    return 1 - (sum(cross) / len(cross))

def find_ovalue_purity(feature, target):
    feature_chunks = [x.tolist() for x in np.array_split(feature, time_res)]
    target_chunks = [x.tolist() for x in np.array_split(target, time_res)]

    cross = []
    # Shoddy method for cross-checking chunks
    for xi, x in enumerate(feature_chunks):
        for yi, y in enumerate(feature_chunks):
            if y > x:  # Avoid repeating the same chunk pair
                xt = target_chunks[xi]
                yt = target_chunks[yi]
                dist = hist_purity(x, y, xt, yt, x_res, weighted, ignore_zero)
                cross.append(dist)
    try:
        return sum(cross) / len(cross)
    except:
        # Will return -1 if there is an error
        return -1


if method == 'inter':
    find_ovalue = find_ovalue_inter
elif method == 'purity':
    find_ovalue = find_ovalue_purity
else:
    print("Method must be set to either 'inter' or 'purity'")

######## EVALUATION #########

# Load dataset into memory however you choose
df_train = pickle.load(open('/media/disk2/dataset-kaggle/avito2018/features_lgb.pickle', 'rb'))
# df_train = df_train[df_train.org==1]

# df_train.drop(['activation_date'  , 
# 'item_id',  
# # 'user_type',                                   
# # 'city',                                        
# # 'region',                                      
# 'image'], axis=1, inplace=True)

# Replace missing values
df_train = df_train.replace([np.inf, -np.inf], np.nan).fillna(0)



print('loaded')
# Function for evaluating features in turn and then writing result to csv
def process(c):
    # Here is the function that evaluates the feature, please provide it with a LIST
    # It returns a float which is the overfitting value
    o = find_ovalue(df_train[c].tolist(), df_train['deal_probability'].tolist())
    print(c.ljust(60) + ' ' + str(o))
    f = open('overfit_valid_mikel_purity.csv', 'a')
    f.write(c+','+str(o)+'\n')
    f.close()

# Multithreading, because I am too impatient to wait 10 minutes :)
pool = Pool(12)
pool.map(process, df_train.columns.tolist())

loaded


/home/xavierc/.local/lib/python3.5/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in true_divide
/home/xavierc/.local/lib/python3.5/site-packages/ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in true_divide
/home/xavierc/.local/lib/python3.5/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in true_divide
/home/xavierc/.local/lib/python3.5/site-packages/ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in true_divide
/home/xavierc/.local/lib/python3.5/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in true_divide
/home/xavierc/.local/lib/python3.5/site-packages/ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in true_divide
/home/xavierc/.local/lib/python3.5/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in true_divide
/home/xavierc/.local/lib/python3.5/site-packages/ipykernel_launcher.py:87: R

price                                                        0.00040390280955161404
user_id_max_days_on_hold_actfrom                             0.0007437003004559255
user_id_min_item_seq_number                                  0.00021766847123623175
user_id_mean_days_on_hold_actfrom                            0.001128535408649699
user_id_min_days_on_hold_actfrom                             0.0007104413705109685
user_id_sum_days_on_hold_actfrom                             0.0002685196200448333


/home/xavierc/.local/lib/python3.5/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in true_divide
/home/xavierc/.local/lib/python3.5/site-packages/ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in true_divide
/home/xavierc/.local/lib/python3.5/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in true_divide
/home/xavierc/.local/lib/python3.5/site-packages/ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in true_divide


image_top_1                                                  0.001300913460544875


...

/home/xavierc/.local/lib/python3.5/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in true_divide
/home/xavierc/.local/lib/python3.5/site-packages/ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in true_divide


user_id_count_item_id                                        0.0002533408725648215
item_seq_number                                              0.00021736279401323846
wh                                                           0.0015157797423491198
blur                                                         0.0005395443465097072
sG                                                           0.0016158472783618693
user_id_sum_days_total_actto                                 0.0002548183903622997
user_id_mean_days_total_actto                                0.000908115122078896
user_id_max_days_total_actto                                 0.0005897028695996238
user_id_min_days_total_actto                                 0.0005822078331418763
mB                                                           0.0015286439537102862
nb_kp2                                                       0.0012137558973686867
mG                                                           0.001983211002720826
user_

/home/xavierc/.local/lib/python3.5/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in true_divide
/home/xavierc/.local/lib/python3.5/site-packages/ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in true_divide


sB                                                           0.0016618478042255505
punctuation_countdescription                                 0.0005654142099675778
mix                                                          0.002182080158047754
words_vs_uniquedescription                                   0.001051150820184609
lower_case_counttitle                                        0.0010048398937885631
num_wordstitle                                               0.0004989591434262566
num_symbolsdescription                                       0.00049861704283375
num_symbolstitle                                             0.0003557809189565019
space_countdescription                                       0.0008224645692384302
num_unique_words_description                                 0.0012543634223312772
char_countdescription                                        0.001387241505865034
word_counttitle                                              0.0004989591434262566
caps_vs_l

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [6]:
df_train['image_top_1'].shape

(1503424,)

In [7]:
df_train['deal_probability'].shape

(1503424,)

In [8]:
process('image_top_1')

ValueError: Length of values does not match length of index